# LM-Evaluation-Harness Quick Start Guide

This notebook demonstrates how to use the lm-evaluation-harness (lm-eval) to evaluate language models using command-line interface.

## Prerequisites

Install lm-eval with the required backends:

In [ ]:
# Install lm-eval with API support
!pip install "lm_eval[api]"

## 1. List Available Tasks

First, let's see what evaluation tasks are available:

In [ ]:
# List all available tasks (showing first 20 lines)
!lm-eval ls tasks | head -20

### Search for Specific Tasks

You can search for specific tasks using grep:

In [ ]:
# Search for MMLU tasks
!lm-eval ls tasks | grep mmlu | head -20

# Search for math-related tasks
!lm-eval ls tasks | grep -i math

# Search for Chinese language tasks
!lm-eval ls tasks | grep zho

## 2. Quick Test with Limited Examples

Before running a full evaluation, it's good practice to test with a small number of examples:

In [ ]:
# Test with 5 examples from hellaswag
# Replace the base_url and model name with your local API endpoint
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-0.5B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks hellaswag \
    --limit 5

## 3. Evaluate on Multiple Tasks

Run evaluation on multiple tasks suitable for API models (generation-based tasks):

In [ ]:
# Evaluate on GSM8K (math reasoning)
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks gsm8k \
    --batch_size 8 \
    --output_path ./results \
    --log_samples

## 4. Evaluate with Configuration File

For more complex evaluations, use a YAML configuration file:

In [ ]:
# Create a configuration file
config = """
model: local-chat-completions
model_args:
  model: Qwen/Qwen2.5-7B-Instruct
  base_url: http://localhost:8000/v1
tasks:
  - gsm8k
  - arc_easy
  - hellaswag
batch_size: 8
output_path: ./results
log_samples: true
"""

with open('eval_config.yaml', 'w') as f:
    f.write(config)

print("Configuration file created!")

In [ ]:
# Run evaluation with config file
!lm-eval --config eval_config.yaml --limit 10

## 5. Comprehensive Evaluation Suite

Run a comprehensive evaluation on multiple benchmarks:

In [ ]:
# Comprehensive evaluation (generation-based tasks for API models)
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks gsm8k,arc_easy,arc_challenge,boolq,piqa \
    --batch_size 8 \
    --output_path ./comprehensive_results \
    --log_samples

## 6. View Results

After evaluation completes, you can view the results:

In [ ]:
import json

# Load and display results
with open('./results/results.json', 'r') as f:
    results = json.load(f)

# Display the results
print("=== Evaluation Results ===")
print(json.dumps(results, indent=2))

# Explain common metrics
print("\n=== Common Output Metrics ===")
print("- acc: Accuracy (proportion of correct answers)")
print("- acc_norm: Normalized accuracy (using length-normalized probabilities)")
print("- exact_match: Exact string match between prediction and reference")
print("- pass@1, pass@10: Percentage of problems solved (for code generation)")
print("- f1: F1 score (harmonic mean of precision and recall)")
print("- bleu, rouge: Text similarity metrics for generation tasks")

## 7. Advanced: Task-Specific Examples

### Mathematics Evaluation (GSM8K with Chain-of-Thought)

In [ ]:
# GSM8K with chain-of-thought reasoning
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks gsm8k_cot \
    --batch_size 8 \
    --output_path ./results/gsm8k_cot

### Multilingual Evaluation

In [ ]:
# Evaluate on Chinese Belebele
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks belebele_zho_Hans \
    --batch_size 8 \
    --output_path ./results/belebele_chinese

### Multiple MMLU Subjects

In [ ]:
# Evaluate on specific MMLU subjects
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks mmlu_abstract_algebra,mmlu_anatomy,mmlu_astronomy \
    --batch_size 8 \
    --output_path ./results/mmlu_subset

## 8. Caching and Resume

Use caching to resume interrupted evaluations:

In [ ]:
# Run with caching enabled
!lm-eval --model local-chat-completions \
    --model_args model=Qwen/Qwen2.5-7B-Instruct,base_url=http://localhost:8000/v1 \
    --tasks gsm8k \
    --batch_size 8 \
    --use_cache ./cache \
    --output_path ./results

## Tips and Best Practices

1. **Always test first**: Use `--limit 5` or `--limit 10` to verify your setup before running full evaluations
2. **Save results**: Use `--output_path` and `--log_samples` for reproducibility
3. **Choose appropriate tasks**: Refer to the complete task list in the documentation for detailed task information
4. **Monitor resources**: Large evaluations can take time; monitor with `htop` or `nvidia-smi`
5. **Use caching**: Enable `--use_cache` for long evaluations that might be interrupted
6. **Batch size**: Adjust `--batch_size` based on your API rate limits and model capacity
7. **API configuration**: Ensure your local model service is running and accessible at the `base_url` you specify

## Resources

- **Complete Task Documentation**: See the main documentation for a comprehensive list of all evaluation tasks and their capabilities
- **lm-eval Documentation**: https://github.com/EleutherAI/lm-evaluation-harness/tree/main/docs
- **GitHub Repository**: https://github.com/EleutherAI/lm-evaluation-harness